In [5]:
import cv2
import mediapipe as mp
import numpy as np

In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
def calculate_angle(a, b, c):
    """
    计算以点 b 为顶点的角度
    参数 a, b, c 为二维坐标 [x, y]
    """
    a = np.array(a)  # 第一点
    b = np.array(b)  # 中间点（顶点）
    c = np.array(c)  # 第三点

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [11]:
cap = cv2.VideoCapture(0)

In [10]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 将 BGR 转换为 RGB，并关闭写入提高效率
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # 恢复图像写入，并转换回 BGR 以供 OpenCV 使用
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # 获取检测到的所有关键点
            landmarks = results.pose_landmarks.landmark

            # 以左肩、左肘、左手腕为例计算左肘角度
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

            # 在画面上显示角度值
            h, w, _ = image.shape
            elbow_coords = tuple((np.array(left_elbow) * [w, h]).astype(int))
            cv2.putText(image, str(int(angle)),
                        elbow_coords,
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # 简单判断：如果角度小于30或大于160，提示动作错误
            if angle < 30 or angle > 160:
                cv2.putText(image, "Incorrect movement", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Good", (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        except Exception as e:
            # 若未检测到人体，则跳过当前帧
            pass

        # 绘制人体姿态连接线
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # 显示处理后的视频画面
        cv2.imshow('Rehab Training', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

I0000 00:00:1741090557.995729  516605 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1741090558.083640  540444 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741090558.102217  540445 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741090558.120025  540442 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-03-04 20:15:58.673 python[85169:516605] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-04 20:15:58.673 python[85169:516605] +[IMKInputSession subclass]: chose IMKInputSession_Modern


KeyboardInterrupt: 